In [1]:
# Load patient data and individual annotations
import datetime as dt
from pathlib import Path
import pandas as pd
from tqdm.notebook import tqdm

## pls excuse
import os, sys
up1 = os.path.abspath('..'); sys.path.insert(0, up1)
## pls excuse

from prediction import loadData as data

# redefine for your data location
DATA_DIR = Path(os.getcwd()).parent  / 'data' / 'assets' / 'europace' / 'physionet.org/files/ltafdb/1.0.0'

# stitching parameters
STITCH_LENGTH_SECONDS=10 #if end of annotation exceeds


def collectAllRhythmsWithinStitch(rhythm='(AFIB', stitch_length = dt.timedelta(seconds=5)):
    global DATA_DIR
    allIDs = data.getAllRecordIDs(europaceDownloadDir=DATA_DIR)
    result = list()
    for id in tqdm(allIDs, total=len(allIDs)):
        #collect all afib annotations for patient
        annotationsForPatient: pd.DataFrame = data.getRhythmAnnotations(id, rhythmToCollect=rhythm)
        #stitch baby
        annotationsForPatient.sort_values(by='start', inplace=True)
        preStitched = annotationsForPatient.copy()
        postStitched = {
            'start': list(),
            'stop': list(),
            'id': list(),
            'basetime': list()
        }
        i = 0
        broke=False
        while (i < len(annotationsForPatient)):
            knitStart = annotationsForPatient.iloc[i, :]['start']
            knitStop = curStop = annotationsForPatient.iloc[i, :]['stop']
            if (i+1 >= len(annotationsForPatient)):
                broke=True
                break
            nextStart = annotationsForPatient.iloc[i+1, :]['start']
            while ((nextStart - curStop) < stitch_length):
                i += 1
                if (i+1 >= len(annotationsForPatient)):
                    broke=True
                    break
                knitStop = curStop = annotationsForPatient.iloc[i, :]['stop']
                nextStart = annotationsForPatient.iloc[i+1, :]['start']
            #time to add entry as a post stitched episode
            postStitched['start'].append(knitStart)
            postStitched['stop'].append(knitStop)
            postStitched['id'].append(id)
            postStitched['basetime'].append(annotationsForPatient.iloc[i-1, :]['basetime'])
            i+=1
        if (broke):
            #time to add entry as a post stitched episode
            postStitched['start'].append(knitStart)
            postStitched['stop'].append(knitStop)
            postStitched['id'].append(id)
            postStitched['basetime'].append(annotationsForPatient.iloc[i-1, :]['basetime'])
        result.append(pd.DataFrame(postStitched))

    return pd.concat(result)

stitchedAnnotations_10sec = collectAllRhythmsWithinStitch(stitch_length=dt.timedelta(seconds=10))
stitchedAnnotations_5sec = collectAllRhythmsWithinStitch(stitch_length=dt.timedelta(seconds=5))
stitchedAnnotations_1sec = collectAllRhythmsWithinStitch(stitch_length=dt.timedelta(seconds=1))

  0%|          | 0/84 [00:00<?, ?it/s]

  0%|          | 0/84 [00:00<?, ?it/s]

  0%|          | 0/84 [00:00<?, ?it/s]

In [ ]:

from prediction.loadData import getRhythmAnnotations, getAllRecordIDs

# utility functions defined for dataframe of annotations: uniquePatients(df), numberOfEvents(df), and filterAnnotationsByLength(df, length)
## i.e., print(uniquePatients(filtered), numberOfEvents(filterAnnotationsByLength(filtered, length=dt.timedelta(seconds=60))))
uniquePatients = lambda df: len(df['id'].unique())
numberOfEvents = lambda df: len(df)
def filterAnnotationsByLength(df, length = dt.timedelta(seconds=30)):
    return df[(df['stop']-df['start']) > length]

allIdsRaw = list()
for id in getAllRecordIDs():
    allIdsRaw.append(getRhythmAnnotations(id))
raw_ltafdb_annotations = pd.concat(allIdsRaw)


In [2]:
# Sydney sync:
## even if our annotation extractions are correct, it seems like some annotations (i.e. changing a rhythm 3 times in a second?) are implausible
## proportion of time the annotation says during a stitch difference --> over 50% necessary, higher is better
## patient_id, start, end, start1, end1, 
import pandas as pd
def proportionOfStitchedEpisodesInAfib(stitchedAnnotations: pd.DataFrame, rawAnnotations: pd.DataFrame):
    rawAnnotationsWithinStitch = lambda df: rawAnnotations[(df['id'] == rawAnnotations['id']) & (df['start'] <= rawAnnotations['start']) & (rawAnnotations['stop'] <= df['stop'])]
    runRawSum = dt.timedelta(seconds=0)
    runStitchSum = dt.timedelta(seconds=0)
    for i, annotation in stitchedAnnotations.iterrows():
        runStitchSum += annotation['stop'] - annotation['start']
        #collect corresponding rawAnnotations and sum their durations
        correspondingRawAnnotations = rawAnnotationsWithinStitch(annotation)
        for i, rawAnn in correspondingRawAnnotations.iterrows():
            runRawSum += rawAnn['stop'] - rawAnn['start']
    return runRawSum / runStitchSum
proportion_10sec = proportionOfStitchedEpisodesInAfib(stitchedAnnotations_10sec, raw_ltafdb_annotations)
proportion_5sec =  proportionOfStitchedEpisodesInAfib(stitchedAnnotations_5sec, raw_ltafdb_annotations)
proportion_1sec =  proportionOfStitchedEpisodesInAfib(stitchedAnnotations_1sec, raw_ltafdb_annotations)



NameError: name 'raw_ltafdb_annotations' is not defined